1. **Indlæsning af Data:**
    ```python
    dataset = pd.read_csv('../modified_dataset.csv', sep=',', encoding='utf-8')
    ```
    - Data indlæses fra en CSV-fil uden at specificere kolonnenavne, da CSV-filen indeholder overskrifter.

2. **Konvertering af 'Incidents'-kolonnen til Numerisk:**
    ```python
    dataset['Incidents'] = pd.to_numeric(dataset['Incidents'], errors='coerce')
    ```
    - Kolonnen 'Incidents' konverteres til numerisk datatype, og eventuelle konverteringsfejl håndteres ved at sætte dem til NaN.

3. **Fjernelse af Rækker med Ikke-Numeriske Værdier:**
    ```python
    dataset = dataset.dropna(subset=['Incidents'])
    ```
    - Evt Rækker, som er 'NULL', fjernes.

4. **Sikring af at 'Incidents'-kolonnen er af Integer-Type:**
    ```python
    dataset['Incidents'] = dataset['Incidents'].astype(int)
    ```
    - Kolonnen 'Incidents' sikres at være af integer-type.

5. **Fjernelse af Rækker med 0 Hændelser:**
    ```python
    dataset = dataset[dataset['Incidents'] > 0]
    ```
    - Rækker, hvor hændelsestallet er 0 og der derved ikke er sket noget uheld, fjernes fra datasættet.

6. **Udvidelse af DataFrame:**
    ```python
    repeated_dataset = dataset.loc[dataset.index.repeat(dataset['Incidents'])]
    ```
    - DataFrame udvides ved at gentage rækkerne baseret på værdien i antal 'Incidents'-kolonnen.

7. **Fjernelse af 'Incidents'-kolonnen:**
    ```python
    repeated_dataset.drop(columns=['Incidents'], inplace=True)
    ```
    - 'Incidents'-kolonnen fjernes, da den ikke længere er nødvendig.

8. **Output og Gemme det Transformerede Datasæt:**
    ```python
    print(repeated_dataset.head())
    repeated_dataset.to_csv('../expanded_dataset.csv', index=False)
    ```
    - Resultatet vises for at kontrollere den korrekte transformation, og det transformerede datasæt gemmes i en ny CSV-fil som vi nu kan bruge.

In [2]:
import pandas as pd

# Load data from CSV without specifying column names if the CSV includes headers
dataset = pd.read_csv('../modified_dataset.csv', sep=',', encoding='utf-8')

# Convert 'Incidents' column to numeric, coercing errors
dataset['Incidents'] = pd.to_numeric(dataset['Incidents'], errors='coerce')

# Drop any rows that could not be converted to numbers (if there were non-numeric values)
dataset = dataset.dropna(subset=['Incidents'])

# Ensure 'Incidents' column is of integer type
dataset['Incidents'] = dataset['Incidents'].astype(int)

# Remove rows where the 'Incidents' count is 0
dataset = dataset[dataset['Incidents'] > 0]

# Explode the DataFrame to repeat rows based on the 'Incidents' column
repeated_dataset = dataset.loc[dataset.index.repeat(dataset['Incidents'])]

# Drop the 'Incidents' column as it is no longer needed
repeated_dataset.drop(columns=['Incidents'], inplace=True)

# Output the result to check
print(repeated_dataset.head())

# Optionally, save the transformed dataset to a new CSV file
repeated_dataset.to_csv('../expanded_dataset.csv', index=False)


  Gender      Age Type of Injury                Area       Type of Vehicle
0   Mænd  0-17 år         Dræbte  Region Hovedstaden  Almindelig personbil
0   Mænd  0-17 år         Dræbte  Region Hovedstaden  Almindelig personbil
8   Mænd  0-17 år         Dræbte  Region Hovedstaden              Knallert
9   Mænd  0-17 år         Dræbte  Region Hovedstaden                 Cykel
9   Mænd  0-17 år         Dræbte  Region Hovedstaden                 Cykel


1. **Import af nødvendige biblioteker:**
    ```python
    from sklearn.model_selection import train_test_split
    from sklearn.tree import DecisionTreeClassifier, export_graphviz
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import classification_report, accuracy_score
    import graphviz
    ```

2. **Forberedelse af data:**
    ```python
    X = repeated_dataset.drop('Type of Injury', axis=1)
    y = repeated_dataset['Type of Injury']
    ```
    - `X` indeholder alle input features undtagen `Type of Injury`, som er vores målvariabel.
    - `y` indeholder kun `Type of Injury`, som vi vil forudsige.

3. **Kodning af kategoriske data:**
    ```python
    label_encoders = {}
    for column in X.columns:
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column])
        label_encoders[column] = le
    ```
    - Hver kolonne i `X` kodet til numeriske værdier ved hjælp af `LabelEncoder`.

4. **Opdeling af data i trænings- og testdatasæt:**
    ```python
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    ```
    - Data opdeles i træningssæt (70%) og testsæt (30%) ved hjælp af `train_test_split`.

5. **Initialisering og træning af beslutningstræ-klassifikatoren:**
    ```python
    tree_classifier = DecisionTreeClassifier(random_state=42)
    tree_classifier.fit(X_train, y_train)
    ```
    - dtree-klassifikatoren initialiseres og trænes med træningsdata.

6. **Forudsigelser og evalueringer:**
    ```python
    y_pred = tree_classifier.predict(X_test)
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Accuracy Score:", accuracy_score(y_test, y_pred))
    ```
    - Klassifikationsrapport og nøjagtighedsscore udskrives for at evaluere modellens præstation på testdata.

7. **Visualisering af beslutningstræet:**
    ```python
    dot_data = export_graphviz(tree_classifier, out_file=None, 
                               feature_names=X.columns,  
                               class_names=tree_classifier.classes_,
                               filled=True, rounded=True, special_characters=True)
    graph = graphviz.Source(dot_data)
    graph.render('decision_tree_improved', format='png')
    ```
    - dtree'et eksporteres som en grafisk repræsentation ved hjælp af `export_graphviz` og gemmes som en PNG-fil.

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import graphviz

# Assuming 'repeated_dataset' is already loaded
# Load your data if it's not loaded
# repeated_dataset = pd.read_csv('../expanded_dataset.csv')

# Prepare the data
X = repeated_dataset.drop('Type of Injury', axis=1)
y = repeated_dataset['Type of Injury']

# Encode categorical data
label_encoders = {}
for column in X.columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    label_encoders[column] = le

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the decision tree classifier
tree_classifier = DecisionTreeClassifier(random_state=42)
tree_classifier.fit(X_train, y_train)

# Predictions and Evaluations
y_pred = tree_classifier.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# Visualize the decision tree
dot_data = export_graphviz(tree_classifier, out_file=None, 
                           feature_names=X.columns,  
                           class_names=tree_classifier.classes_,
                           filled=True, rounded=True, special_characters=True)
graph = graphviz.Source(dot_data)
graph.render('decision_tree_improved', format='png')

Classification Report:
                          precision    recall  f1-score   support

Alvorligt tilskadekomne       0.62      0.82      0.71      1499
                 Dræbte       0.29      0.03      0.05       146
  Lettere tilskadekomne       0.48      0.29      0.36       922

               accuracy                           0.59      2567
              macro avg       0.46      0.38      0.37      2567
           weighted avg       0.55      0.59      0.55      2567

Accuracy Score: 0.5862874951305025


ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

1. **Definition af `predict_injury`-funktionen:**
    ```python
    def predict_injury(gender, age, area, type_of_vehicle):
        # Create a DataFrame for the input features
        input_data = pd.DataFrame({
            'Gender': [gender],
            'Age': [age],
            'Area': [area],
            'Type of Vehicle': [type_of_vehicle]
        })
    ```
    - Funktionen starter med at skabe en DataFrame `input_data` med de fire inputparametre: køn, alder, region og køretøjstype.

2. **Kodning af inputdata:**
    ```python
    # Encode the input data using the same label encoders used for training
    for column, le in label_encoders.items():
        input_data[column] = le.transform(input_data[column])
    ```
    - Hver kolonne i `input_data` bliver kodet til numeriske værdier ved hjælp af de samme `LabelEncoder`-objekter, som blev brugt under træningen af modellen. Dette sikrer konsistens mellem trænings- og forudligelse.

3. **Forudsigelse ved hjælp af beslutningstræ-modellen:**
    ```python
    # Make prediction
    prediction = tree_classifier.predict(input_data)
    ```
    - Den trænede model (`tree_classifier`) bruges til at forudsige typen af skade baseret på de en-kodede inputdata.

4. **Returnering af forudsagt klasse:**
    ```python
    # Get the predicted class label
    predicted_class = prediction[0]
    return predicted_class
    ```
    - Den forudsagte klasse (`predicted_class`) hentes fra resultatet af forudsigelsen og returneres.

5. **Eksempel på brug af funktionen:**
    ```python
    # Example usage:
    predicted_injury = predict_injury('Kvinder', '18-24 år', 'Region Hovedstaden', 'Almindelig personbil')
    print("Predicted Type of Injury:", predicted_injury)
    ```
    - Et eksempel på, hvordan funktionen kan bruges, vises ved at kalde `predict_injury` med specifikke inputværdier og udskrive den forudsagte type skade.

In [3]:
def predict_injury(gender, age, area, type_of_vehicle):
    # Create a DataFrame for the input features
    input_data = pd.DataFrame({
        'Gender': [gender],
        'Age': [age],
        'Area': [area],
        'Type of Vehicle': [type_of_vehicle]
    })
    
    # Encode the input data using the same label encoders used for training
    for column, le in label_encoders.items():
        input_data[column] = le.transform(input_data[column])
    
    # Make prediction
    prediction = tree_classifier.predict(input_data)
    
    # Get the predicted class label
    predicted_class = prediction[0]
    return predicted_class

# Example usage:
predicted_injury = predict_injury('Kvinder', '18-24 år', 'Region Hovedstaden', 'Almindelig personbil')
print("Predicted Type of Injury:", predicted_injury)


Predicted Type of Injury: Alvorligt tilskadekomne


1. **Import af nødvendige moduler:**
    ```python
    from sklearn.model_selection import GridSearchCV
    ```

2. **Definering af parametergrid:**
    ```python
    # Expanded parameter grid
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 5, 10, 15, 20, 25, 30],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 5, 10, 15],
        'max_features': [None, 'sqrt', 'log2', 0.5, 0.75]
    }
    ```
    - Et udvidet parametergrid specificeres for at dække et bredt udvalg af mulige værdier for forskellige hyperparametre. Dette gøres for at se om vi kan få en bedre accuracy.

3. **Opsætning af GridSearchCV:**
    ```python
    # Grid search with more extensive grid
    grid_search = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                               param_grid=param_grid, 
                               cv=5,
                               verbose=2,  # Increase verbosity to see more details during the fitting
                               scoring='accuracy',
                               n_jobs=-1)  # Use all available CPU cores
    ```
    - `GridSearchCV` initialiseres med beslutningstræ-klassifikatoren og det definerede parametergrid. Krydvalidering sættes til 5 folde (cv=5),og scoren er nøjagtighed (`accuracy`). `n_jobs=-1` betyder, at alle tilgængelige CPU-kerner bruges til beregningen.

4. **Fit GridSearchCV:**
    ```python
    # Fit GridSearchCV
    grid_search.fit(X_train, y_train)
    ```
    - Grid Search fit-tes til træningsdatasættet for at finde de bedste hyperparametre.

5. **Output af de bedste parametre og score:**
    ```python
    # Output the best parameters and the best score
    print("Best parameters:", grid_search.best_params_)
    print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
    ```
    - De bedste parametre og den bedste krydsvalideringsscore udskrives.

6. **Evaluering på testdatasættet:**
    ```python
    # Evaluate on the test set using the best parameters
    best_tree = grid_search.best_estimator_
    y_pred = best_tree.predict(X_test)
    print("Test Accuracy with Best Parameters:", accuracy_score(y_test, y_pred))
    ```
    - Den bedste model anvendes på testdatasættet for at evaluere nøjagtigheden.

## Resultater

Efter at have kørt Grid Search med det udvidede parametergrid, blev følgende bedste parametre fundet:
- **Best parameters:** `{'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 10, 'min_samples_split': 2}`
- **Best cross-validation score:** `0.62`
- **Test Accuracy with Best Parameters:** `0.61`

Den optimerede beslutningstræ-model er visualiseret i filen `decision_tree_more_optimized.png`.

Dette viser, hvordan Grid Search kan bruges til at forbedre en model ved at systematisk afprøve forskellige kombinationer af hyperparametre og vælge den bedste baseret på krydsvalideringsscoren. 
Vi fik en smule bedre accuracy. Dog ikke meget. Det betyder måske at modellen ikke kan forbedres mere med det nuværende data, men dette er ikke sikkert.

In [4]:
from sklearn.model_selection import GridSearchCV

# Expanded parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20, 25, 30],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 5, 10, 15],
    'max_features': [None, 'sqrt', 'log2', 0.5, 0.75]
}

# Grid search with more extensive grid
grid_search = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                           param_grid=param_grid, 
                           cv=5,
                           verbose=2,  # Increase verbosity to see more details during the fitting
                           scoring='accuracy',
                           n_jobs=-1)  # Use all available CPU cores

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Output the best parameters and the best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

# Evaluate on the test set using the best parameters
best_tree = grid_search.best_estimator_
y_pred = best_tree.predict(X_test)
print("Test Accuracy with Best Parameters:", accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 1750 candidates, totalling 8750 fits
Best parameters: {'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 10, 'min_samples_split': 2}
Best cross-validation score: 0.62
Test Accuracy with Best Parameters: 0.6073237241916635



1. **Import af nødvendige moduler:**
    ```python
    import graphviz
    from sklearn.tree import export_graphviz
    ```
    - `graphviz` bruges til at generere og visualisere beslutningstræet.
    - `export_graphviz` fra `sklearn.tree` bruges til at eksportere beslutningstræet i DOT-format, som Graphviz kan læse.

2. **Eksport af beslutningstræet til DOT-format:**
    ```python
    # Visualize the optimized decision tree
    dot_data = export_graphviz(best_tree, out_file=None, 
                               feature_names=X.columns,  
                               class_names=best_tree.classes_,
                               filled=True, rounded=True, special_characters=True)  # Control the size as previously discussed
    ```
    - `export_graphviz` funktion eksportere det trænede beslutningstræ (`best_tree`) til DOT-format. Parametrene inkluderer:
      - `feature_names=X.columns`: Navnene på de features, der bruges i modellen.
      - `class_names=best_tree.classes_`: Navnene på klasserne i målsættet.
      - `filled=True`: Noderne fyldes med farver, der repræsenterer forskellige klasser.
      - `rounded=True`: Noderne har afrundede hjørner.
      - `special_characters=True`: Tillader specielle tegn i output.

3. **Generering af grafen og gemme den som en PNG-fil:**
    ```python
    graph = graphviz.Source(dot_data)
    graph_path = graph.render('decision_tree_more_optimized', format='png')
    ```
    - `graphviz.Source` initialiserer en Graphviz kildeobjekt med DOT-dataen (`dot_data`).
    - `graph.render` genererer grafen og gemmer den som en PNG-fil med navnet `decision_tree_more_optimized`.

In [5]:
import graphviz
from sklearn.tree import export_graphviz

# Visualize the optimized decision tree
dot_data = export_graphviz(best_tree, out_file=None, 
                           feature_names=X.columns,  
                           class_names=best_tree.classes_,
                           filled=True, rounded=True, special_characters=True)  # Control the size as previously discussed

graph = graphviz.Source(dot_data)
graph_path = graph.render('decision_tree_more_optimized', format='png')


1. **Import af nødvendige moduler:**
    ```python
    from joblib import dump
    ```
    - `dump` funktionen fra `joblib` modulet bruges til at gemme Python-objekter til en fil.

2. **Gemme den bedste trænede model:**
    ```python
    dump(best_tree, 'decision_tree_model.joblib')
    ```
    - `dump` funktionen gemmer `best_tree` modellen til en fil med navnet `decision_tree_model.joblib`.
    - Filen `decision_tree_model.joblib` kan senere indlæses for at genbruge den trænede model uden at skulle træne den igen.

In [6]:
from joblib import dump

dump(best_tree, 'decision_tree_model.joblib')

['decision_tree_model.joblib']